
Copyright (c) 2018 Leland Stanford Junior University
Copyright (c) 2018 The Regents of the University of California

This file is part of pelicun.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice,
this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice,
this list of conditions and the following disclaimer in the documentation
and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
may be used to endorse or promote products derived from this software without
specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE
LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
POSSIBILITY OF SUCH DAMAGE.

You should have received a copy of the BSD 3-Clause License along with
pelicun. If not, see <http://www.opensource.org/licenses/>.

Contributors:
Adam Zsarnóczay

In [ ]:
"""
This module has classes and methods that control the loss assessment.

.. rubric:: Contents

.. autosummary::

    Assessment
    FEMA_P58_Assessment

"""

In [ ]:
# from .base import *
# from .uq import *
# from .model import *
# from .file_io import *

In [ ]:
class Assessment(object):
    """
    A high-level class that collects features common to all supported loss
    assessment methods. This class will only rarely be called directly when
    using pelicun.
    """

    def __init__(self):

        # initialize the basic data containers
        # inputs
        self._AIM_in = None
        self._EDP_in = None
        self._POP_in = None
        self._FG_in = None

        # random variables and loss model
        self._RV_dict = None # dictionary to store random variables
        self._EDP_dict = None
        self._FG_dict = None

        # results
        self._TIME = None
        self._POP = None
        self._COL = None
        self._ID_dict = None
        self._DMG = None
        self._DV_dict = None
        self._SUMMARY = None

        self._assessment_type = 'generic'

    @property
    def beta_tot(self):
        """
        Calculate the total additional uncertainty for post processing.

        The total additional uncertainty is the squared root of sum of squared
        uncertainties corresponding to ground motion and modeling.

        Returns
        -------
        beta_total: float
            The total uncertainty (logarithmic EDP standard deviation) to add
            to the EDP distribution. Returns None if no additional uncertainty
            is assigned.
        """

        AU = self._AIM_in['general']['added_uncertainty']

        beta_total = 0.
        if AU['beta_m'] is not None:
            beta_total += AU['beta_m'] ** 2.
        if AU['beta_gm'] is not None:
            beta_total += AU['beta_gm'] ** 2.

        # if no uncertainty is assigned, we return None
        if beta_total == 0:
            beta_total = None
        else:
            beta_total = np.sqrt(beta_total)

        return beta_total

    def read_inputs(self, path_DL_input, path_EDP_input, verbose=False):
        """
        Read and process the input files to describe the loss assessment task.

        Parameters
        ----------
        path_DL_input: string
            Location of the Damage and Loss input file. The file is expected to
            be a JSON with data stored in a standard format described in detail
            in the Input section of the documentation.
        path_EDP_input: string
            Location of the EDP input file. The file is expected to follow the
            output formatting of Dakota. The Input section of the documentation
            provides more information about the expected formatting.
        verbose: boolean, default: False
            If True, the method echoes the information read from the files.
            This can be useful to ensure that the information in the file is
            properly read by the method.
        """

        # read SimCenter inputs -----------------------------------------------
        # BIM file
        self._AIM_in = read_SimCenter_DL_input(
            path_DL_input, assessment_type=self._assessment_type,
            verbose=verbose)

        # EDP file
        if self._hazard == 'EQ':
            self._EDP_in = read_SimCenter_EDP_input(
                path_EDP_input,
                units=dict(PID=1.,
                           PFA=self._AIM_in['units']['acceleration']),
                verbose=verbose)
        elif self._hazard == 'HU':
            self._EDP_in = read_SimCenter_EDP_input(
                path_EDP_input, EDP_kinds=('PWS',),
                units=dict(PWS=self._AIM_in['units']['speed']),
                verbose=verbose)

    def define_random_variables(self):
        """
        Define the random variables used for loss assessment.

        """
        pass

    def define_loss_model(self):
        """
        Create the stochastic loss model based on the inputs provided earlier.

        """
        pass

    def calculate_damage(self):
        """
        Characterize the damage experienced in each random event realization.

        """
        self._ID_dict = {}

    def calculate_losses(self):
        """
        Characterize the consequences of damage in each random event realization.

        """
        self._DV_dict = {}

    def write_outputs(self):
        """
        Export the results.

        """
        pass

    def _create_RV_demands(self):

        # Unlike other random variables, the demand RV is based on raw data.

        # First, collect the raw values from the EDP dict...
        demand_data = []
        d_tags = []
        detection_limits = []
        collapse_limits = []
        GI = self._AIM_in['general']
        s_edp_keys = sorted(self._EDP_in.keys())
        for d_id in s_edp_keys:
            d_list = self._EDP_in[d_id]
            for i in range(len(d_list)):
                demand_data.append(d_list[i]['raw_data'])
                d_tags.append(str(d_id) +
                              '-LOC-' + str(d_list[i]['location']) +
                              '-DIR-' + str(d_list[i]['direction']))
                det_lim = GI['detection_limits'][d_id]
                if det_lim is None:
                    det_lim = np.inf
                if GI['response']['EDP_dist_basis'] == 'non-collapse results':
                    coll_lim = GI['collapse_limits'][d_id]
                    if coll_lim is None:
                        coll_lim = np.inf
                elif GI['response']['EDP_dist_basis'] == 'all results':
                    coll_lim = np.inf

                detection_limits.append([0., det_lim])
                collapse_limits.append([0., coll_lim])

        detection_limits = np.transpose(np.asarray(detection_limits))
        collapse_limits = np.transpose(np.asarray(collapse_limits))
        demand_data = np.transpose(np.asarray(demand_data))

        # If more than one sample is available...
        if demand_data.shape[0] > 1:

            # Second, we discard the collapsed EDPs if the fitted distribution shall
            # represent non-collapse EDPs.
            EDP_filter = np.all([np.all(demand_data > collapse_limits[0], axis=1),
                                 np.all(demand_data < collapse_limits[1], axis=1)],
                                axis=0)
            demand_data = demand_data[EDP_filter]

            # Third, we censor the EDPs that are beyond the detection limit.
            EDP_filter = np.all([np.all(demand_data > detection_limits[0], axis=1),
                                 np.all(demand_data < detection_limits[1], axis=1)],
                                axis=0)
            censored_count = len(EDP_filter) - sum(EDP_filter)
            demand_data = demand_data[EDP_filter]
            demand_data = np.transpose(demand_data)

            # Fourth, we create the random variable
            demand_RV = RandomVariable(ID=200, dimension_tags=d_tags,
                                       raw_data=demand_data,
                                       detection_limits=detection_limits,
                                       censored_count=censored_count
                                       )

            # And finally, if requested, we fit a multivariate lognormal or a
            # truncated multivariate lognormal distribution to the censored raw
            # data.
            target_dist = GI['response']['EDP_distribution']

            if target_dist == 'lognormal':
                demand_RV.fit_distribution('lognormal')
            elif target_dist == 'truncated lognormal':
                demand_RV.fit_distribution('lognormal', collapse_limits)

        # This is a special case when only a one sample is provided.
        else:
            # TODO: what to do when the sample is larger than the collapse or detection limit and when truncated distribution is prescribed

            # Since we only have one data point, the best we can do is assume
            # it is the median of the multivariate distribution. The dispersion
            # is assumed to be negligible.
            dim = len(demand_data[0])
            if dim > 1:
                sig = np.abs(demand_data[0])*1e-6
                rho = np.zeros((dim,dim))
                np.fill_diagonal(rho, 1.0)
                COV = np.outer(sig,sig) * rho
            else:
                COV = np.abs(demand_data[0][0])*(1e-6)**2.0

            demand_RV = RandomVariable(ID=200, dimension_tags=d_tags,
                                       distribution_kind='lognormal',
                                       theta=demand_data[0],
                                       COV=COV)

        # To consider additional uncertainty in EDPs, we need to redefine the
        # random variable. If the EDP distribution is set to 'empirical' then
        # adding uncertainty by increasing its variance is not possible.
        if ((self.beta_tot is not None) and
            (GI['response']['EDP_distribution'] != 'empirical')):
            # determine the covariance matrix with added uncertainty
            if demand_RV.COV.shape != ():
                sig_mod = np.sqrt(demand_RV.sig ** 2. + self.beta_tot ** 2.)
                COV_mod = np.outer(sig_mod, sig_mod) * demand_RV.corr
            else:
                COV_mod = np.sqrt(demand_RV.COV**2. + self.beta_tot**2.)

            # redefine the random variable
            demand_RV = RandomVariable(
                ID=200,
                dimension_tags=demand_RV.dimension_tags,
                distribution_kind=demand_RV.distribution_kind,
                theta=demand_RV.theta,
                COV=COV_mod)

        return demand_RV

In [ ]:
class HAZUS_Assessment_BLZ(Assessment):
    """
    An Assessment class that implements the damage and loss assessment method
    following the HAZUS Technical Manual and the HAZUS software.

    Parameters
    ----------
    hazard:  {'EQ', 'HU'}
        Identifies the type of hazard. EQ corresponds to earthquake, HU
        corresponds to hurricane.
        default: 'EQ'.
    inj_lvls: int
        Defines the discretization used to describe the severity of injuries.
        The HAZUS earthquake methodology uses 4 levels.
        default: 4
    """
    def __init__(self, hazard='EQ', inj_lvls = 4):
        super(HAZUS_Assessment, self).__init__()

        self._inj_lvls = inj_lvls
        self._hazard = hazard
        self._assessment_type = 'HAZUS_{}'.format(hazard)

    def read_inputs(self, path_DL_input, path_EDP_input, verbose=False):
        """
        Read and process the input files to describe the loss assessment task.

        Parameters
        ----------
        path_DL_input: string
            Location of the Damage and Loss input file. The file is expected to
            be a JSON with data stored in a standard format described in detail
            in the Input section of the documentation.
        path_EDP_input: string
            Location of the EDP input file. The file is expected to follow the
            output formatting of Dakota. The Input section of the documentation
            provides more information about the expected formatting.
        verbose: boolean, default: False
            If True, the method echoes the information read from the files.
            This can be useful to ensure that the information in the file is
            properly read by the method.

        """

        super(HAZUS_Assessment, self).read_inputs(path_DL_input,
                                                  path_EDP_input, verbose)

        # assume that the asset is a building
        # TODO: If we want to apply HAZUS to non-building assets, several parts of this methodology need to be extended.
        BIM = self._AIM_in

        # read component and population data ----------------------------------
        # components
        self._FG_in = read_component_DL_data(
            self._AIM_in['data_sources']['path_CMP_data'], BIM['components'],
            assessment_type=self._assessment_type, verbose=verbose)

        # population (if needed)
        if self._AIM_in['decision_variables']['injuries']:
            POP = read_population_distribution(
                self._AIM_in['data_sources']['path_POP_data'],
                BIM['general']['occupancy_type'],
                assessment_type=self._assessment_type,
                verbose=verbose)

            POP['peak'] = BIM['general']['population']
            self._POP_in = POP

    def define_random_variables(self):
        """
        Define the random variables used for loss assessment.

        Following the HAZUS methodology, only the groups of parameters below
        are considered random. Correlations within groups are not considered
        because each Fragility Group has only one Performance Group with a
        in this implementation.

        1. Demand (EDP) distribution

        Describe the uncertainty in the demands. Unlike other random variables,
        the EDPs are characterized by the EDP input data provided earlier. All
        EDPs are handled in one multivariate lognormal distribution. If more
        than one sample is provided, the distribution is fit to the EDP data.
        Otherwise, the provided data point is assumed to be the median value
        and the additional uncertainty prescribed describes the dispersion. See
        _create_RV_demands() for more details.

        2. Fragility EDP limits

        Describe the uncertainty in the EDP limit that corresponds to
        exceedance of each Damage State. EDP limits are grouped by Fragility
        Groups. See _create_RV_fragilities() for details.

        """
        super(HAZUS_Assessment, self).define_random_variables()

        # create the random variables -----------------------------------------
        self._RV_dict = {}

        # fragilities 300
        s_fg_keys = sorted(self._FG_in.keys())
        for c_id, c_name in enumerate(s_fg_keys):
            comp = self._FG_in[c_name]

            self._RV_dict.update({
                'FR-' + c_name:
                    self._create_RV_fragilities(c_id, comp,'PG')})

        # demands 200
        self._RV_dict.update({'EDP': self._create_RV_demands()})

        # sample the random variables -----------------------------------------
        realization_count = self._AIM_in['general']['realizations']
        is_coupled = self._AIM_in['general']

        s_rv_keys = sorted(self._RV_dict.keys())
        for r_i in s_rv_keys:
            rv = self._RV_dict[r_i]
            if rv is not None:
                rv.sample_distribution(
                    sample_size=realization_count, preserve_order=is_coupled)

    def define_loss_model(self):
        """
        Create the stochastic loss model based on the inputs provided earlier.

        Following the HAZUS methodology, the component assemblies specified in
        the Damage and Loss input file are used to create Fragility Groups.
        Each Fragility Group corresponds to one assembly that represents every
        component of the given type in the structure. See
        _create_fragility_groups() for more details about the creation of
        Fragility Groups.

        """
        super(HAZUS_Assessment, self).define_loss_model()

        # fragility groups
        self._FG_dict = self._create_fragility_groups()

        # demands
        self._EDP_dict = dict(
            [(tag, RandomVariableSubset(self._RV_dict['EDP'], tags=tag))
             for tag in self._RV_dict['EDP']._dimension_tags])

    def calculate_damage(self):
        """
        Characterize the damage experienced in each random event realization.

        First, the time of the event (month, weekday/weekend, hour) is randomly
        generated for each realization. Given the event time, the number of
        people present at each floor of the building is calculated.

        Next, the quantities of components in each damage state are estimated.
        See _calc_damage() for more details on damage estimation.

        """
        super(HAZUS_Assessment, self).calculate_damage()

        # event time - month, weekday, and hour realizations
        self._TIME = self._sample_event_time()

        # if we are interested in injuries...
        if self._AIM_in['decision_variables']['injuries']:
            # get the population conditioned on event time
            self._POP = self._get_population()

        # collapses are handled as the ultimate DS in HAZUS
        self._ID_dict.update({'collapse': []})

        # select the non-collapse cases for further analyses
        non_collapsed_IDs = self._TIME.index.values.astype(int)
        self._ID_dict.update({'non-collapse': non_collapsed_IDs})

        # damage in non-collapses
        self._DMG = self._calc_damage()

    def calculate_losses(self):
        """
        Characterize the consequences of damage in each random event realization.

        For the sake of efficiency, only the decision variables requested in
        the input file are estimated. The following consequences are handled by
        this method for a HAZUS assessment:

        Reconstruction time and cost
        Get a cost and time estimate for each Damage State in each Performance
        Group. For more information about estimating reconstruction cost and
        time see _calc_repair_cost_and_time() methods.

        Injuries
        The number of injuries are based on the probability of injuries of
        various severity specified in the component data file. For more
        information about estimating injuries _calc_non_collapse_injuries.

        """
        super(HAZUS_Assessment, self).calculate_losses()
        DVs = self._AIM_in['decision_variables']

        # reconstruction cost and time
        if DVs['rec_cost'] or DVs['rec_time']:
            # all damages are considered repairable in HAZUS
            repairable_IDs = self._ID_dict['non-collapse']
            self._ID_dict.update({'repairable': repairable_IDs})
            self._ID_dict.update({'irrepairable': []})

            # reconstruction cost and time for repairable cases
            DV_COST, DV_TIME = self._calc_repair_cost_and_time()

            if DVs['rec_cost']:
                self._DV_dict.update({'rec_cost': DV_COST})

            if DVs['rec_time']:
                self._DV_dict.update({'rec_time': DV_TIME})

        # injuries due to collapse
        if DVs['injuries']:
            # there are no separate collapse cases in HAZUS

            # injuries in non-collapsed cases
            DV_INJ_dict = self._calc_non_collapse_injuries()

            # store result
            self._DV_dict.update({'injuries': DV_INJ_dict})

    def aggregate_results(self):
        """

        Returns
        -------

        """

        DVs = self._AIM_in['decision_variables']

        MI_raw = [
            ('event time', 'month'),
            ('event time', 'weekday?'),
            ('event time', 'hour'),
            ('reconstruction', 'cost'),
        ]

        if DVs['rec_time']:
            MI_raw += [
                ('reconstruction', 'time'),
            ]

        if DVs['injuries']:
            MI_raw += [
                ('inhabitants', ''),
                ('injuries', 'sev. 1'),
                ('injuries', 'sev. 2'),
                ('injuries', 'sev. 3'),
                ('injuries', 'sev. 4'),
            ]

        ncID = self._ID_dict['non-collapse']
        colID = self._ID_dict['collapse']
        if DVs['rec_cost'] or DVs['rec_time']:
            repID = self._ID_dict['repairable']
            irID = self._ID_dict['irrepairable']

        MI = pd.MultiIndex.from_tuples(MI_raw)

        SUMMARY = pd.DataFrame(np.empty((
            self._AIM_in['general']['realizations'],
            len(MI))), columns=MI)
        SUMMARY[:] = np.NaN

        # event time
        for prop in ['month', 'weekday?', 'hour']:
            offset = 0
            if prop == 'month':
                offset = 1
            SUMMARY.loc[:, ('event time', prop)] = \
                self._TIME.loc[:, prop] + offset

        # inhabitants
        if DVs['injuries']:
            SUMMARY.loc[:, ('inhabitants', '')] = self._POP.sum(axis=1)

        # reconstruction cost
        if DVs['rec_cost']:
            repl_cost = self._AIM_in['general']['replacement_cost']

            SUMMARY.loc[ncID, ('reconstruction', 'cost')] = \
                self._DV_dict['rec_cost'].sum(axis=1)
            #SUMMARY.loc[:, ('reconstruction', 'cost')] *= repl_cost

        # reconstruction time
        if DVs['rec_time']:
            SUMMARY.loc[ncID, ('reconstruction', 'time')] = \
                self._DV_dict['rec_time'].sum(axis=1)

        # injuries
        if DVs['injuries']:
            for sev_id in range(4):
                sev_tag = 'sev. {}'.format(sev_id+1)
                SUMMARY.loc[ncID, ('injuries', sev_tag)] = \
                    self._DV_dict['injuries'][sev_id].sum(axis=1)

        self._SUMMARY = SUMMARY.dropna(axis=1, how='all')

    def _create_RV_fragilities(self, c_id, comp, rho_fr):
        """

        Parameters
        ----------
        c_id
        comp
        rho_fr

        Returns
        -------

        """

        # prepare the basic multivariate distribution data for one component subgroup considering all damage states
        d_theta, d_sig, d_tag, d_distr_kind = [np.array([]) for i in range(4)]

        s_dsg_keys = sorted(comp['DSG_set'].keys())
        for d_id in s_dsg_keys:
            DSG = comp['DSG_set'][d_id]
            d_theta = np.append(d_theta, DSG['theta'])
            d_sig = np.append(d_sig, DSG['sig'])
            d_tag = np.append(d_tag, comp['ID'] + '-' + str(d_id))
            d_distr_kind = np.append(d_distr_kind, DSG['distribution_kind'])
        dims = len(d_theta)

        # get the total number of random variables for this fragility group
        # TODO: add the possibility of multiple locations and directions
        #rv_count = len(comp['locations']) * len(comp['directions']) * dims
        rv_count = sum([len(csg_w) for csg_w in comp['csg_weights']]) * dims

        # create the (empty) input arrays for the RV
        c_theta = np.zeros(rv_count)
        c_tag = np.empty(rv_count, dtype=object)
        c_sig = np.zeros(rv_count)
        c_distr_kind = np.empty(rv_count, dtype=object)

        pos_id = 0
        #for l_id in comp['locations']:
        #    # for each location-direction pair)
        #    for d_id, __ in enumerate(comp['directions']):
        #        # for each component-subgroup
        #        c_theta[pos_id:pos_id + dims] = d_theta
        #        c_sig[pos_id:pos_id + dims] = d_sig
        #        c_tag[pos_id:pos_id + dims] = [
        #            t + '-LOC-{}-CSG-{}'.format(l_id, d_id) for t in d_tag]
        #        c_distr_kind[pos_id:pos_id + dims] = d_distr_kind
        #        pos_id += dims

        for l_id, d_id, csg_list in zip(comp['locations'], comp['directions'],
                                        comp['csg_weights']):
            # for each location-direction pair)
            for csg_id, __ in enumerate(csg_list):
                # for each component-subgroup
                c_theta[pos_id:pos_id + dims] = d_theta
                c_sig[pos_id:pos_id + dims] = d_sig
                c_tag[pos_id:pos_id + dims] = [
                    t + '-LOC-{}-DIR-{}-CSG-{}'.format(l_id, d_id, csg_id)
                    for t in d_tag]
                c_distr_kind[pos_id:pos_id + dims] = d_distr_kind
                pos_id += dims

        # create the covariance matrix
        #c_rho = self._create_correlation_matrix(rho_fr, c_target=c_id,
        #                                        include_DSG=True,
        #                                        include_CSG=True)
        c_rho = np.ones((rv_count, rv_count))
        c_COV = np.outer(c_sig, c_sig) * c_rho

        if c_tag.size > 0:
            fragility_RV = RandomVariable(ID=300 + c_id,
                                          dimension_tags=c_tag,
                                          distribution_kind=c_distr_kind,
                                          theta=c_theta,
                                          COV=c_COV)
        else:
            fragility_RV = None

        return fragility_RV

    def _create_fragility_groups(self):

        RVd = self._RV_dict
        DVs = self._AIM_in['decision_variables']

        # use the building replacement cost to calculate the absolute
        # reconstruction cost for component groups
        repl_cost = self._AIM_in['general']['replacement_cost']

        # create a list for the fragility groups
        FG_dict = dict()

        s_fg_keys = sorted(self._FG_in.keys())
        for c_id in s_fg_keys:
            comp = self._FG_in[c_id]

            FG_ID = len(FG_dict.keys()) + 1

            # create a list for the performance groups
            performance_groups = []

            # one group for each of the stories prescribed by the user
            PG_locations = comp['locations']
            PG_directions = comp['directions']
            PG_csg_lists = comp['csg_weights']
            for loc, dir_, csg_list in zip(PG_locations, PG_directions,
                                           PG_csg_lists):
                PG_ID = 1000 * FG_ID + 10 * loc + dir_

                # get the quantity
                QNT = None
                #QNT = RandomVariableSubset(
                #    RVd['QNT'],
                #    tags=[c_id + '-QNT-' + str(loc) + '-' + str(dir_), ])

                # create the damage objects
                # consequences are calculated on a performance group level

                # create a list for the damage state groups and their tags
                DSG_list = []
                d_tags = []
                s_dsg_keys = sorted(comp['DSG_set'].keys())
                for dsg_i, DSG_ID in enumerate(s_dsg_keys):
                    DSG = comp['DSG_set'][DSG_ID]
                    d_tags.append(c_id + '-' + DSG_ID)

                    # create a list for the damage states
                    DS_set = []

                    s_ds_keys = sorted(DSG['DS_set'].keys())
                    for ds_i, DS_ID in enumerate(s_ds_keys):
                        DS = DSG['DS_set'][DS_ID]

                        # create the consequence functions
                        # note: consequences in HAZUS are conditioned on
                        # damage with no added uncertainty

                        if DVs['rec_cost']:
                            data = DS['repair_cost']
                            f_median = prep_constant_median_DV(data*repl_cost)
                            CF_cost = ConsequenceFunction(
                                DV_median=f_median,
                                DV_distribution=None)
                        else:
                            CF_cost = None

                        if DVs['rec_time'] and ('repair_time' in DS.keys()):
                            data = DS['repair_time']
                            f_median = prep_constant_median_DV(data)
                            CF_time = ConsequenceFunction(
                                DV_median=f_median,
                                DV_distribution=None)
                        else:
                            CF_time = None

                        # note: no red tag in HAZUS assessments

                        if (DVs['injuries']) and ('injuries' in DS.keys()):
                            CF_inj_set = []
                            for inj_i, theta in enumerate(
                                DS['injuries']):
                                if theta > 0.:
                                    f_median = prep_constant_median_DV(
                                        theta)
                                    CF_inj_set.append(ConsequenceFunction(
                                        DV_median=f_median,
                                        DV_distribution=None))
                                else:
                                    CF_inj_set.append(None)
                        else:
                            CF_inj_set = [None, ]

                        DS_set.append(DamageState(ID=ds_i + 1,
                                                  description=DS[
                                                      'description'],
                                                  weight=DS['weight'],
                                                  repair_cost_CF=CF_cost,
                                                  reconstruction_time_CF=CF_time,
                                                  injuries_CF_set=CF_inj_set
                                                  ))

                    # add the DSG to the list
                    DSG_list.append(DamageStateGroup(ID=dsg_i + 1,
                                                     DS_set=DS_set,
                                                     DS_set_kind=DSG[
                                                         'DS_set_kind']
                                                     ))

                # create the fragility functions
                FF_set = []
                #CSG_this = np.where(comp['directions'] == dir_)[0]
                #PG_weights = np.asarray(comp['csg_weights'])[CSG_this]
                # normalize the weights
                #PG_weights /= sum(PG_weights)
                for csg_id, __ in enumerate(csg_list):
                    # assign the appropriate random variable to the fragility
                    # function
                    ff_tags = [t + '-LOC-{}-DIR-{}-CSG-{}'.format(loc, dir_,
                                                                  csg_id)
                               for t in d_tags]
                    EDP_limit = RandomVariableSubset(RVd['FR-' + c_id],
                                                     tags=ff_tags)
                    FF_set.append(FragilityFunction(EDP_limit))

                # create the performance group
                PG = PerformanceGroup(ID=PG_ID,
                                      location=loc,
                                      quantity=QNT,
                                      fragility_functions=FF_set,
                                      DSG_set=DSG_list,
                                      csg_weights=csg_list,
                                      direction=dir_
                                      )
                performance_groups.append(PG)

            # create the fragility group
            FG = FragilityGroup(ID=FG_ID,
                                #kind=comp['kind'],
                                demand_type=comp['demand_type'],
                                performance_groups=performance_groups,
                                directional=comp['directional'],
                                correlation=comp['correlation'],
                                demand_location_offset=comp['offset'],
                                incomplete=comp['incomplete'],
                                name=str(FG_ID) + ' - ' + comp['ID'],
                                description=comp['description']
                                )

            FG_dict.update({comp['ID']: FG})

        return FG_dict

    def _sample_event_time(self):

        sample_count = self._AIM_in['general']['realizations']

        # month - uniform distribution over [0,11]
        month = np.random.randint(0, 12, size=sample_count)

        # weekday - binomial with p=5/7
        weekday = np.random.binomial(1, 5. / 7., size=sample_count)

        # hour - uniform distribution over [0,23]
        hour = np.random.randint(0, 24, size=sample_count)

        data = pd.DataFrame(data={'month'   : month,
                                  'weekday?': weekday,
                                  'hour'    : hour},
                            dtype=int)

        return data

    def _get_population(self):
        """
        Use the population characteristics to generate random population samples.

        Returns
        -------

        """
        POPin = self._POP_in
        TIME = self._TIME

        POP = pd.DataFrame(
            np.ones((len(TIME.index), len(POPin['peak']))) * POPin['peak'],
            columns=['LOC' + str(loc + 1)
                     for loc in range(len(POPin['peak']))])

        weekdays = TIME[TIME['weekday?'] == 1].index
        weekends = TIME[~TIME.index.isin(weekdays)].index

        for col in POP.columns.values:
            POP.loc[weekdays, col] = (
                POP.loc[weekdays, col] *
                np.array(POPin['weekday']['daily'])[
                    TIME.loc[weekdays, 'hour'].values.astype(int)] *
                np.array(POPin['weekday']['monthly'])[
                    TIME.loc[weekdays, 'month'].values.astype(int)])

            POP.loc[weekends, col] = (
                POP.loc[weekends, col] *
                np.array(POPin['weekend']['daily'])[
                    TIME.loc[weekends, 'hour'].values.astype(int)] *
                np.array(POPin['weekend']['monthly'])[
                    TIME.loc[weekends, 'month'].values.astype(int)])

        return POP

    def _calc_damage(self):

        ncID = self._ID_dict['non-collapse']
        NC_samples = len(ncID)
        DMG = pd.DataFrame()

        s_fg_keys = sorted(self._FG_dict.keys())
        for fg_id in s_fg_keys:
            FG = self._FG_dict[fg_id]

            PG_set = FG._performance_groups

            DS_list = []
            for DSG in PG_set[0]._DSG_set:
                for DS in DSG._DS_set:
                    DS_list.append(str(DSG._ID) + '-' + str(DS._ID))
            d_count = len(DS_list)

            MI = pd.MultiIndex.from_product([[FG._ID, ],
                                             [pg._ID for pg in PG_set],
                                             DS_list],
                                            names=['FG', 'PG', 'DS'])

            FG_damages = pd.DataFrame(np.zeros((NC_samples, len(MI))),
                                      columns=MI,
                                      index=ncID)

            for pg_i, PG in enumerate(PG_set):

                PG_ID = PG._ID
                if PG._quantity is not None:
                    PG_qnt = PG._quantity.samples.loc[ncID]
                else:
                    PG_qnt = pd.DataFrame(np.ones(NC_samples),index=ncID)

                # get the corresponding demands
                demand_ID = (FG._demand_type +
                             '-LOC-' + str(PG._location + FG._demand_location_offset) +
                             '-DIR-' + str(PG._direction))
                if demand_ID in self._EDP_dict.keys():
                    EDP_samples = self._EDP_dict[demand_ID].samples.loc[ncID]
                else:
                    # If the required demand is not available, then we are most
                    # likely analyzing a 3D structure using results from a 2D
                    # simulation. The best thing we can do in that particular
                    # case is to use the EDP from the 1 direction for all other
                    # directions.
                    demand_ID = (FG._demand_type +
                                 '-LOC-' + str(PG._location + FG._demand_location_offset) + '-DIR-1')
                    EDP_samples = self._EDP_dict[demand_ID].samples.loc[ncID]

                csg_w_list = PG._csg_weights

                for csg_i, csg_w in enumerate(csg_w_list):
                    DSG_df = PG._FF_set[csg_i].DSG_given_EDP(EDP_samples)

                    for DSG in PG._DSG_set:
                        in_this_DSG = DSG_df[DSG_df.values == DSG._ID].index
                        if DSG._DS_set_kind == 'single':
                            DS = DSG._DS_set[0]
                            DS_tag = str(DSG._ID) + '-' + str(DS._ID)
                            FG_damages.loc[in_this_DSG,
                                           (FG._ID, PG_ID, DS_tag)] += csg_w
                        elif DSG._DS_set_kind == 'mutually exclusive':
                            DS_weights = [DS._weight for DS in DSG._DS_set]
                            DS_RV = RandomVariable(
                                ID=-1, dimension_tags=['me_DS', ],
                                distribution_kind='multinomial',
                                p_set=DS_weights)
                            DS_df = DS_RV.sample_distribution(
                                len(in_this_DSG)) + 1
                            for DS in DSG._DS_set:
                                DS_tag = str(DSG._ID) + '-' + str(DS._ID)
                                in_this_DS = DS_df[DS_df.values == DS._ID].index
                                FG_damages.loc[in_this_DSG[in_this_DS],
                                               (FG._ID, PG_ID, DS_tag)] += csg_w
                        elif DSG._DS_set_kind == 'simultaneous':
                            DS_weights = [DS._weight for DS in DSG._DS_set]
                            DS_df = np.random.uniform(
                                size=(len(in_this_DSG), len(DS_weights)))
                            which_DS = DS_df < DS_weights
                            any_DS = np.any(which_DS, axis=1)
                            no_DS_ids = np.where(any_DS == False)[0]

                            while len(no_DS_ids) > 0:
                                DS_df_add = np.random.uniform(
                                    size=(len(no_DS_ids), len(DS_weights)))
                                which_DS_add = DS_df_add < DS_weights
                                which_DS[no_DS_ids] = which_DS_add

                                any_DS = np.any(which_DS_add, axis=1)
                                no_DS_ids = no_DS_ids[
                                    np.where(any_DS == False)[0]]

                            for ds_i, DS in enumerate(DSG._DS_set):
                                DS_tag = str(DSG._ID) + '-' + str(DS._ID)
                                in_this_DS = which_DS[:, ds_i]
                                FG_damages.loc[in_this_DSG[in_this_DS], (
                                    FG._ID, PG_ID, DS_tag)] += csg_w

                        else:
                            raise ValueError(
                                "Unknown damage state type: {}".format(
                                    DSG._DS_set_kind)
                            )

                FG_damages.iloc[:, pg_i * d_count:(pg_i + 1) * d_count] = \
                    FG_damages.mul(PG_qnt.iloc[:, 0], axis=0)

            DMG = pd.concat((DMG, FG_damages), axis=1)

        DMG.index = ncID

        # sort the columns to enable index slicing later
        DMG = DMG.sort_index(axis=1, ascending=True)

        return DMG

    def _calc_repair_cost_and_time(self):

        idx = pd.IndexSlice
        DVs = self._AIM_in['decision_variables']

        DMG_by_FG_and_DS = self._DMG.groupby(level=[0, 2], axis=1).sum()

        repID = self._ID_dict['repairable']
        REP_samples = len(repID)
        DV_COST = pd.DataFrame(np.zeros((REP_samples, len(self._DMG.columns))),
                               columns=self._DMG.columns, index=repID)
        DV_TIME = deepcopy(DV_COST)

        s_fg_keys = sorted(self._FG_dict.keys())
        for fg_id in s_fg_keys:
            FG = self._FG_dict[fg_id]

            PG_set = FG._performance_groups

            DS_list = self._DMG.loc[:, idx[FG._ID, PG_set[0]._ID, :]].columns
            DS_list = DS_list.levels[2][DS_list.codes[2]].values

            for pg_i, PG in enumerate(PG_set):

                PG_ID = PG._ID

                for d_i, d_tag in enumerate(DS_list):
                    dsg_i = int(d_tag[0]) - 1
                    ds_i = int(d_tag[-1]) - 1

                    DS = PG._DSG_set[dsg_i]._DS_set[ds_i]

                    TOT_qnt = DMG_by_FG_and_DS.loc[repID, (FG._ID, d_tag)]
                    PG_qnt = self._DMG.loc[repID,
                                           (FG._ID, PG_ID, d_tag)]

                    # repair cost
                    if DVs['rec_cost']:
                        COST_samples = DS.unit_repair_cost(quantity=TOT_qnt)
                        if COST_samples is not None:
                            DV_COST.loc[:,
                            (FG._ID, PG_ID, d_tag)] = COST_samples * PG_qnt

                    if DVs['rec_time']:
                        # repair time
                        TIME_samples = DS.unit_reconstruction_time(
                            quantity=TOT_qnt)
                        if TIME_samples is not None:
                            DV_TIME.loc[:,
                            (FG._ID, PG_ID, d_tag)] = TIME_samples * PG_qnt

        # sort the columns to enable index slicing later
        if DVs['rec_cost']:
            DV_COST = DV_COST.sort_index(axis=1, ascending=True)
        else:
            DV_COST = None
        if DVs['rec_time']:
            DV_TIME = DV_TIME.sort_index(axis=1, ascending=True)
        else:
            DV_TIME = None

        return DV_COST, DV_TIME

    def _calc_non_collapse_injuries(self):

        idx = pd.IndexSlice

        ncID = self._ID_dict['non-collapse']
        NC_samples = len(ncID)
        DV_INJ_dict = dict([(i, pd.DataFrame(np.zeros((NC_samples,
                                                       len(self._DMG.columns))),
                                             columns=self._DMG.columns,
                                             index=ncID))
                            for i in range(self._inj_lvls)])
        s_fg_keys = sorted(self._FG_dict.keys())
        for fg_id in s_fg_keys:
            FG = self._FG_dict[fg_id]

            PG_set = FG._performance_groups

            DS_list = self._DMG.loc[:, idx[FG._ID, PG_set[0]._ID, :]].columns
            DS_list = DS_list.levels[2][DS_list.codes[2]].values

            for pg_i, PG in enumerate(PG_set):

                PG_ID = PG._ID

                for d_i, d_tag in enumerate(DS_list):
                    dsg_i = int(d_tag[0]) - 1
                    ds_i = int(d_tag[-1]) - 1

                    DS = PG._DSG_set[dsg_i]._DS_set[ds_i]

                    P_affected = self._POP.loc[ncID]

                    QNT = self._DMG.loc[:, (FG._ID, PG_ID, d_tag)]

                    # estimate injuries
                    for i in range(self._inj_lvls):
                        INJ_samples = DS.unit_injuries(severity_level=i,
                                                       sample_size=NC_samples)
                        if INJ_samples is not None:
                            P_aff_i = P_affected.loc[:,
                                      'LOC{}'.format(PG._location)]
                            INJ_i = INJ_samples * P_aff_i * QNT
                            DV_INJ_dict[i].loc[:,
                            (FG._ID, PG_ID, d_tag)] = INJ_i

        # remove the useless columns from DV_INJ
        for i in range(self._inj_lvls):
            DV_INJ = DV_INJ_dict[i]
            DV_INJ_dict[i] = DV_INJ.loc[:, (DV_INJ != 0.0).any(axis=0)]

        # sort the columns to enable index slicing later
        for i in range(self._inj_lvls):
            DV_INJ_dict[i] = DV_INJ_dict[i].sort_index(axis=1, ascending=True)

        return DV_INJ_dict